# Good boy LLM


## imports

In [6]:
from accelerate.test_utils.scripts.external_deps.test_ds_alst_ulysses_sp import tokenizer

from dataset import load_dataset
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from huggingface_hub import hf_hub_download, notebook_login
import numpy as np
import einops
import textwrap
from typing import Literal
import plotly.express as px
from functools import partial
import dataclasses
from IPython.display import display, HTML
import gc
import pandas as pd
from safetensors.torch import load_file
import torch
import torch.nn as nn

from sae_lens import SAE

from enum import Enum


class MODEL_SIZE(Enum):
    M270 = "270m"
    B1 = "1b"
    B3 = "3b"
    B7 = "7b"


# load model and tokenizer
torch.set_grad_enabled(False)  # avoid blowing up mem
if torch.backends.mps.is_available():
    device = "mps"

### huggingface login

In [17]:
notebook_login()

## Model, SAE and Tokenizer

### loading model

In [22]:
model = AutoModelForCausalLM.from_pretrained(
        "google/gemma-3-270m-pt",
        device_map='auto',
    )

OSError: google/gemma-3-270m-pt is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

### loading tokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-3-270m-it",
)

RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-69592bf6-7218db42708356311c13a429;0e02fb59-ca4f-4730-87a0-816a15481486)

Entry Not Found for url: https://huggingface.co/api/models/google/gemma-3-270m-it/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"

### l

In [4]:
model, sae, tokenizer = get_test_components(model_size=MODEL_SIZE.M270, instructions_tuned=True)

RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-69592a32-4a5151917f27bf9e46b93a79;b1a90bd6-6c1f-4e0d-a238-ffddb42de112)

Entry Not Found for url: https://huggingface.co/api/models/google/gemma-3-270m-it/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"

## Loading dataset


In [1]:
dataset = load_dataset("HuggingFaceH4/instruction-dataset")["test"]
for data in dataset:
    print(data)
    break

NameError: name 'load_dataset' is not defined

In [ ]:
ds = load_dataset("HuggingFaceH4/instruction-dataset")["test"]

new_column = ["You did a very good job! Thank you i am happy!"]*len(ds)
ds = ds.add_column("positive feedback", new_column)


In [ ]:
chat_history = [
    {"role": "user", "content": "Hi, can you explain quantum mechanics?"},
    {"role": "model", "content": "Certainly! Quantum mechanics is..."},
    {"role": "user", "content": "Can you simplify that for a 5-year-old?"}  # The new prompt
]
# 2. Convert to the correct string format
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# 3. Pass to the model
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))